LOAD DATA 

In [52]:
import numpy as np
import matplotlib.pyplot as plt
data_path ='data\Task 2\sample_1a2bc3012c9defca.npz'


In [53]:
info = np.load(data_path,allow_pickle=True)
all_agent_trajs        = info['all_agent']    
all_agent_trajs.shape  #[24 agents, 91 timesteps the interval between each timestep is 0.1 second,\
    #10 denotes [center_x, center_y, center_z, boundingbox_x, boundingbox_y, boundingbox_z, heading, vel_x, vel_y, valid] ]
#the valid flag is either 0 or 1. 1 means the valid, 0 means invalid
obj_types              = info['object_type']
lane_polylines         = info['lane']           #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
road_polylines         = info['road_polylines'] #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
crosswalk_polylines    = info['crosswalk']      #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
speed_bump_polylines   = info['speed_bump']     #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
stop_signs_polylines   = info['stop_sign']      #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
drive_way_polylines    = info['drive_way']      #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]


We select the 11th timestep as the current timestep 

In [54]:
all_agent_current = all_agent_trajs[:,10]
all_agent_current.shape

(24, 10)

In [55]:
#ground truth for the future 1s,2s,3s can be easily derived by :
predict_horizon = 10 #20,30
all_gt_future=all_agent_trajs[:,11:11+predict_horizon]
all_gt_future.shape

(24, 10, 10)

Index of target agent to predict

In [56]:
tracks        = info['predict_list']  
tracks
agents_to_predict = all_agent_trajs[tracks]
agents_to_predict.shape

(8, 91, 10)

The homework should be done on these 8 agents.

# Q1.Using constant velocity model to predict the future (1s,2s,3s) trajectories of the target agent and calculate the ADE and FDE.  

This code is designed for trajectory prediction and evaluation within a multi-object tracking scenario. It specifically aims to predict the future positions of selected agents using a constant velocity model and then evaluates the accuracy of these predictions against ground truth data. The evaluation metrics used are the Average Displacement Error (ADE) and the Final Displacement Error (FDE). Here's a brief overview of its components:

### Data Loading and Preprocessing
- The code starts by loading a dataset from a NumPy compressed file (`.npz`), which includes data for various agents and environmental features.
- It extracts the indices of specific agents to be predicted (`tracks`) and their corresponding trajectory data (`agents_to_predict`).
- It defines prediction horizons (in seconds) and translates them into timesteps, considering a timestep interval of 0.1 seconds.
- Ground truth data for the future positions of these agents are extracted for each defined prediction horizon to later compare with the predicted positions.

### Constant Velocity Model Prediction
- A function `predict_trajectory_constant_velocity` is defined to predict the future positions of the agents based on the assumption of constant velocity. For each agent and each prediction horizon, it calculates future positions and stores them in an array.
- The prediction considers the agent's last known position and velocity and extrapolates future positions linearly.

### Calculation of ADE and FDE
- The Average Displacement Error (ADE) is calculated as the mean Euclidean distance between the predicted and actual positions across all predicted timesteps for each agent, averaged over all agents.
- The Final Displacement Error (FDE) is calculated as the Euclidean distance between the predicted and actual positions at the final predicted timestep, again averaged over all agents.
- Two functions, `calculate_ade` and `calculate_fde`, implement these calculations by comparing the predicted trajectories against the ground truth for each prediction horizon.

### Output
- Finally, the script prints out the calculated ADE and FDE values for each prediction horizon, providing a quantitative assessment of the prediction accuracy.



In [64]:
# Data Loading and Preprocessing
# Load the dataset from a NumPy compressed file (.npz), which contains multiple arrays with data for agents, object types, and environmental features.
data_path = 'data/Task 2/sample_1a2bc3012c9defca.npz'  
info = np.load(data_path, allow_pickle=True)

# Extract agents that are specified for prediction based on an index list provided within the dataset.
tracks = info['predict_list']
agents_to_predict = info['all_agent'][tracks]

# Prediction horizons are defined here, indicating how far into the future we want to predict (in seconds), converted into timesteps by multiplying with 10 (assuming a 0.1-second interval per step).
predict_horizons = [10, 20, 30]

# Extract the ground truth data for future positions of agents for comparison with the predicted trajectories.
all_gt_futures = {horizon: info['all_agent'][tracks, 11:11+horizon] for horizon in predict_horizons}

# Constant Velocity Model Prediction
# This function predicts the future trajectory of each agent based on constant velocity assumption for specified horizons.
def predict_trajectory_constant_velocity(agent_positions, predict_horizons):
    predicted_trajectories = {}
    for horizon in predict_horizons:
        predictions = np.zeros((agent_positions.shape[0], horizon, 2))  # Prepare a zero-filled array for storing predictions.
        for i, agent in enumerate(agent_positions):
            velocity_x = agent[10, 7]  # Extract the x-component of velocity.
            velocity_y = agent[10, 8]  # Extract the y-component of velocity.
            for t in range(horizon):
                # Calculate future position based on constant velocity.
                future_x = agent[10, 0] + velocity_x * (t + 1) * 0.1
                future_y = agent[10, 1] + velocity_y * (t + 1) * 0.1
                predictions[i, t] = [future_x, future_y]
        predicted_trajectories[horizon] = predictions
    return predicted_trajectories

predicted_trajectories = predict_trajectory_constant_velocity(agents_to_predict, predict_horizons)

# Calculate ADE and FDE
# ADE is the average Euclidean distance between predicted positions and ground truth positions across all timesteps and agents.
# FDE is the Euclidean distance between the predicted and actual positions at the final timestep.
def calculate_ade(predicted_trajectories, ground_truth):
    ade_values = {}
    for horizon, predictions in predicted_trajectories.items():
        total_error = 0
        for i in range(predictions.shape[0]):
            for t in range(horizon):
                pred_pos = predictions[i, t]
                true_pos = ground_truth[horizon][i, t, :2]
                total_error += np.linalg.norm(pred_pos - true_pos)
        ade_values[horizon] = total_error / (predictions.shape[0] * horizon)
    return ade_values

def calculate_fde(predicted_trajectories, ground_truth):
    fde_values = {}
    for horizon, predictions in predicted_trajectories.items():
        total_error = 0
        for i in range(predictions.shape[0]):
            pred_pos = predictions[i, -1]
            true_pos = ground_truth[horizon][i, -1, :2]
            total_error += np.linalg.norm(pred_pos - true_pos)
        fde_values[horizon] = total_error / predictions.shape[0]
    return fde_values

ade_values = calculate_ade(predicted_trajectories, all_gt_futures)
fde_values = calculate_fde(predicted_trajectories, all_gt_futures)

# Print the calculated ADE and FDE values for each prediction horizon.
print("ADE values:", ade_values)
print("FDE values:", fde_values)


ADE values: {10: 0.2508319052301113, 20: 0.6408690170888223, 30: 1.287818545500022}
FDE values: {10: 0.5150812726918637, 20: 1.5804000554098399, 30: 3.540838200019236}


# Q2.Using constant acceleration model to predict the future (1s,2s,3s) trajectories of the target agent and calculate the ADE and FDE. 

This code snippet is designed to predict the future positions of multiple agents using a Constant Acceleration Model (CAM) and then evaluate the accuracy of these predictions by calculating the Average Displacement Error (ADE) and Final Displacement Error (FDE). Here's a step-by-step explanation:

### Estimating Acceleration (`estimate_acceleration`)
- **Purpose**: Calculates the acceleration of an agent based on the change in its velocity between the last two timesteps.
- **Input**: Trajectory data of an agent, where each row corresponds to a timestep and includes various data such as position and velocity.
- **Process**: The function retrieves the agent's velocities at the last and second-to-last timesteps, calculates the change in velocity (delta velocity), and divides it by the time interval (0.1 seconds) to estimate the acceleration in both the x and y directions.
- **Output**: A 2-element numpy array representing the estimated acceleration (`accel_x`, `accel_y`).

### Predicting Trajectories (`predict_trajectory_constant_acceleration`)
- **Purpose**: Predicts future positions for each agent over specified prediction horizons using a constant acceleration model.
- **Input**: 
  - `agents`: A numpy array containing the trajectory data for multiple agents.
  - `predict_horizons`: A list of integers representing the prediction horizons in timesteps.
- **Process**: 
  - For each prediction horizon, the function initializes an array to store predicted positions for all agents.
  - It iterates over each agent, using their current position and velocity along with the estimated acceleration to predict future positions at each timestep within the prediction horizon.
  - The future positions are calculated using the constant acceleration equation, taking into account the acceleration in both x and y directions.
- **Output**: A dictionary where keys are prediction horizons and values are numpy arrays containing the predicted positions for all agents.

### Evaluating Predictions (ADE and FDE Calculations)
- After predicting the trajectories, the code calculates two metrics to evaluate the accuracy of these predictions against the ground truth data:
  - **Average Displacement Error (ADE)**: The average Euclidean distance between the predicted and actual positions across all timesteps and agents.
  - **Final Displacement Error (FDE)**: The Euclidean distance between the predicted and actual positions at the final timestep.
- The ADE and FDE values are calculated for each prediction horizon and printed out to provide insights into the model's performance.


In [65]:
# Function to estimate the acceleration of an agent based on the change in its velocity.
def estimate_acceleration(agent):
    """
    Estimates the acceleration of an agent.

    Parameters:
    - agent: A numpy array representing the agent's trajectory data, where each row is a timestep,
      and columns include positions, velocities, etc.

    Returns:
    - A numpy array representing the acceleration (accel_x, accel_y).
    """
    # Retrieve the agent's velocities at the last and second to last timesteps.
    velocity_last = agent[-1, 7:9]  # Last known velocity components (vel_x, vel_y).
    velocity_second_last = agent[-2, 7:9]  # Second to last known velocity components (vel_x, vel_y).
    
    # Calculate the acceleration as the change in velocity divided by the time interval (0.1 seconds here).
    acceleration = (velocity_last - velocity_second_last) / 0.1
    return acceleration

# Function to predict future positions using the Constant Acceleration Model.
def predict_trajectory_constant_acceleration(agents, predict_horizons):
    """
    Predicts future trajectories for multiple agents using a constant acceleration model.

    Parameters:
    - agents: A numpy array of agents, where each agent's trajectory data is included.
    - predict_horizons: A list of integers representing prediction horizons in timesteps.

    Returns:
    - A dictionary where keys are prediction horizons and values are numpy arrays of predicted positions.
    """
    predicted_trajectories = {}
    for horizon in predict_horizons:
        # Initialize an array to hold predicted positions for all agents at this horizon.
        predictions = np.zeros((agents.shape[0], horizon, 2))  # x, y positions
        
        for num_agent, agent in enumerate(agents):
            # Use the agent's current position as the starting point for predictions.
            last_pos_x, last_pos_y = agent[10, 0], agent[10, 1]
            # Retrieve the current velocity components.
            vel_x, vel_y = agent[10, 7], agent[10, 8]
            # Estimate the agent's acceleration.
            acceleration = estimate_acceleration(agent)
            accel_x, accel_y = acceleration

            # Predict future positions at each timestep within the horizon.
            for timestep_idx in range(horizon):
                # Calculate future position using the constant acceleration equation.
                curr_pos_x = 0.5 * accel_x * (0.1 ** 2) + vel_x * 0.1 + last_pos_x
                curr_pos_y = 0.5 * accel_y * (0.1 ** 2) + vel_y * 0.1 + last_pos_y
                
                # Update the positions for the next iteration.
                last_pos_x, last_pos_y = curr_pos_x, curr_pos_y
                
                # Store the predicted position.
                predictions[num_agent, timestep_idx, :] = [curr_pos_x, curr_pos_y]

        predicted_trajectories[horizon] = predictions
    return predicted_trajectories

# Predict trajectories for all agents at specified horizons.
predicted_trajectories = predict_trajectory_constant_acceleration(agents_to_predict, predict_horizons)

# Calculate and print ADE and FDE for the predictions compared to ground truth.
ade_values = calculate_ade(predicted_trajectories, all_gt_futures)
fde_values = calculate_fde(predicted_trajectories, all_gt_futures)

print("ADE values for Constant Acceleration Model:", ade_values)
print("FDE values for Constant Acceleration Model:", fde_values)


ADE values for Constant Acceleration Model: {10: 0.233451117162717, 20: 0.5806123339154756, 30: 1.1694128572687688}
FDE values for Constant Acceleration Model: {10: 0.4739612837201345, 20: 1.4155385495650414, 30: 3.2543015922438867}


# Using Constant Turn Rate and Velocity(CTRV) model to predict the future (1s,2s,3s) trajectories of the target agent and calculate the ADE and FDE. 

This code implements a trajectory prediction and evaluation framework using the Constant Turn Rate and Velocity (CTRV) model, aimed at predicting the future positions of moving agents (e.g., vehicles) over specified prediction horizons. It then assesses the accuracy of these predictions using Average Displacement Error (ADE) and Final Displacement Error (FDE) metrics. Here's a detailed breakdown:

### Estimating Turn Rate (`estimate_turn_rate`)
- **Purpose**: Computes the rate of change in the heading angle (turn rate, $\omega$) for an agent between the last two time steps, assuming uniform time intervals.
- **Process**: The function calculates the difference in heading angles between two consecutive time steps and divides it by the time interval (0.1 seconds) to obtain the turn rate.
- **Output**: The estimated turn rate ($\omega$) in radians per second.

### Calculating ADE and FDE (`calculate_ade_fde_per_agent`)
- **ADE Calculation**: Measures the average Euclidean distance between the predicted positions and the actual positions (ground truth) across all prediction time steps for each agent.
- **FDE Calculation**: Measures the Euclidean distance between the predicted and actual positions at the final prediction time step for each agent.
- **Purpose**: These metrics evaluate the accuracy of the trajectory predictions, with lower values indicating higher prediction accuracy.

### Predicting Trajectories (`predict_trajectory_ctrv`)
- **Purpose**: Predicts future trajectories for agents using the CTRV model, which accounts for both linear velocity and turn rate, making it suitable for scenarios involving curved paths.
- **Process**:
  - For each agent identified by indices in `tracks`, the function calculates predicted positions at each time step within the specified prediction horizons (e.g., 1s, 2s, 3s) based on their current state (position, velocity, heading) and estimated turn rate.
  - If the turn rate ($\omega$) is non-zero, it predicts the positions using CTRV equations that incorporate both linear motion and rotation. For a zero turn rate, it simplifies to linear motion prediction.
- **Output**: A dictionary mapping each prediction horizon to another dictionary containing ADE and FDE metrics for each agent, indicating the prediction accuracy at that horizon.

### Evaluation and Display
- After predicting trajectories and calculating ADE and FDE, the script iterates through the results to display these metrics for each agent at each prediction horizon. This output helps understand the model's performance and its potential applicability to real-world scenarios involving moving agents.


In [59]:
def estimate_turn_rate(agent):
    # Estimate the turn rate (omega) based on the change in heading angle between two time steps
    heading_initial = agent[-3, 6]  # Heading angle at the second to last time step
    heading_final = agent[-2, 6]  # Heading angle at the last time step
    delta_heading = heading_final - heading_initial
    delta_time = 0.1  # Time interval between the steps, assuming 0.1 seconds
    omega = delta_heading / delta_time
    return omega

def calculate_ade_fde_per_agent(predicted, ground_truth):
    # Calculate ADE (Average Displacement Error) for each agent
    ade = np.mean(np.sqrt(np.sum((predicted - ground_truth) ** 2, axis=2)), axis=1)
    # Calculate FDE (Final Displacement Error) for each agent
    fde = np.sqrt(np.sum((predicted[:, -1, :] - ground_truth[:, -1, :]) ** 2, axis=1))
    return ade, fde

def predict_trajectory_ctrv(agents, predict_horizons, all_gt_futures, tracks):
    results = {}

    for horizon in predict_horizons:
        predictions = np.zeros((agents.shape[0], horizon, 2))  # Initialize predictions
        ground_truth = all_gt_futures[:, 11:11+horizon, :2]  # Extract ground truth for the horizon

        agent_results = {}
        for idx, agent_idx in enumerate(tracks):  # Use enumerate to get both index and agent ID
            agent = agents[idx]
            x, y, theta = agent[10, :3]  # Initial position and heading
            velocity = np.sqrt(agent[10, 7]**2 + agent[10, 8]**2)  # Velocity magnitude
            omega = estimate_turn_rate(agent)  # Turn rate

            for t in range(1, horizon + 1):
                delta_t = t * 0.1  # Time step
                if omega != 0:  # Turning motion
                    x += velocity / omega * (np.sin(theta + omega * delta_t) - np.sin(theta))
                    y += velocity / omega * (np.cos(theta) - np.cos(theta + omega * delta_t))
                else:  # Straight line motion
                    x += velocity * delta_t * np.cos(theta)
                    y += velocity * delta_t * np.sin(theta)
                predictions[idx, t-1, :] = [x, y]

            ade, fde = calculate_ade_fde_per_agent(predictions[idx:idx+1], ground_truth[agent_idx:agent_idx+1, :horizon, :2])
            agent_results[agent_idx] = {'ADE': ade[0], 'FDE': fde[0]}

        results[horizon] = agent_results

    return results

# Assuming the rest of your setup is correct
results = predict_trajectory_ctrv(agents_to_predict, predict_horizons, all_agent_trajs, tracks)

# Display the results
for horizon, agents_data in results.items():
    print(f"Horizon {horizon/10}s:")
    for agent_idx, metrics in agents_data.items():
        print(f"Agent {agent_idx}: ADE = {metrics['ADE']}, FDE = {metrics['FDE']}")


Horizon 1.0s:
Agent 22: ADE = 1.8780475830425658, FDE = 4.924128541702375
Agent 15: ADE = 15.743120299389238, FDE = 36.97064485885443
Agent 4: ADE = 9.490049969407375, FDE = 22.414874090342
Agent 5: ADE = 18.873035745388567, FDE = 45.635511228213
Agent 9: ADE = 26.05552960224906, FDE = 61.77005825046284
Agent 14: ADE = 11.746672276672097, FDE = 27.741904348411772
Agent 6: ADE = 7.656089478338845, FDE = 18.236311868305016
Agent 7: ADE = 6.155643836927794, FDE = 14.5244352504396
Horizon 2.0s:
Agent 22: ADE = 7.164391495628128, FDE = 20.1760633427626
Agent 15: ADE = 49.50594041760547, FDE = 129.37575137982435
Agent 4: ADE = 30.104587805681724, FDE = 79.0159742591865
Agent 5: ADE = 62.73259368968233, FDE = 168.05899139695364
Agent 9: ADE = 83.37181996196507, FDE = 219.66051754085348
Agent 14: ADE = 37.226177778345644, FDE = 97.60067998111265
Agent 6: ADE = 24.68070602119446, FDE = 65.18260702719998
Agent 7: ADE = 19.48549480994416, FDE = 51.06096126894323
Horizon 3.0s:
Agent 22: ADE = 15.9

# Comparison of results constant velocity and constant acceleration 

The outputs provided give insight into the performance of two trajectory prediction models - the Constant Velocity Model (Q1) and the Constant Acceleration Model (Q2) - based on Average Displacement Error (ADE) and Final Displacement Error (FDE) metrics across three prediction horizons (1s, 2s, 3s). Here's a comparison and discussion of the observations:

### Average Displacement Error (ADE)

- **Constant Velocity Model** shows increasing ADE values with longer prediction horizons: 0.2508 (1s), 0.6409 (2s), 1.2878 (3s). This trend indicates that prediction accuracy decreases as the prediction horizon extends, which is expected due to accumulating uncertainties in motion over time.
- **Constant Acceleration Model** also exhibits a similar trend with ADE values: 0.2335 (1s), 0.5806 (2s), 1.1694 (3s). The ADE values are consistently lower across all horizons compared to the Constant Velocity Model, suggesting improved prediction accuracy when accounting for acceleration.

### Final Displacement Error (FDE)

- **Constant Velocity Model** shows FDE values increasing with the prediction horizon: 0.5151 (1s), 1.5804 (2s), 3.5408 (3s). This aligns with the expectation that errors in position prediction accumulate over time.
- **Constant Acceleration Model** reports slightly lower FDE values than the Constant Velocity Model at all horizons: 0.4740 (1s), 1.4155 (2s), 3.2543 (3s). This suggests that incorporating acceleration into the prediction model offers a more accurate estimation of the agent's final position, especially for longer prediction horizons.

### Observations

- **Improved Accuracy with Acceleration**: The Constant Acceleration Model generally provides more accurate predictions (lower ADE and FDE) compared to the Constant Velocity Model. This improvement underscores the importance of accounting for changes in velocity when predicting future positions, particularly in scenarios where agents frequently adjust their speed.
- **Increasing Errors Over Time**: Both models exhibit increasing errors with longer prediction horizons, which is expected in trajectory prediction tasks. The accumulation of errors over time highlights the challenge of predicting movements accurately in dynamic environments.
- **Model Selection Based on Scenario**: While the Constant Acceleration Model shows better overall performance, the choice between these models should consider the specific requirements and constraints of the application scenario. For example, the Constant Velocity Model may suffice in scenarios with mostly uniform motion, offering simpler computations.

In [60]:
sdc_current_state = all_agent_trajs[tracks[0]][11]
ax = plt.gca()
fig = plt.gcf()
fig.set_facecolor('xkcd:grey') 
ax.set_facecolor('xkcd:grey')
for polyline in road_polylines:
    map_type = polyline[0,6]
    if map_type == 6:
        plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='dashed', linewidth=1)
    elif map_type == 7:
        plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='solid', linewidth=1)
    elif map_type == 8:
        plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='solid', linewidth=1)
    elif map_type == 9:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dashed', linewidth=1)
    elif map_type == 10:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dashed', linewidth=1)
    elif map_type == 11:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='solid', linewidth=1)
    elif map_type == 12:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='solid', linewidth=1)
    elif map_type == 13:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dotted', linewidth=1)
    elif map_type == 15:
        plt.plot(polyline[:, 0], polyline[:, 1], 'k', linewidth=1)
    elif map_type == 16:
        plt.plot(polyline[:, 0], polyline[:, 1], 'k', linewidth=1)
ax.axis([-70+ sdc_current_state[0], 70+ sdc_current_state[0], -70+ sdc_current_state[1], 70 + sdc_current_state[1]])

filename = './viz.png' 
plt.savefig(filename)       
plt.close()